# 🧪 Poldantic: Mixed and Optional Types Example

This notebook shows how `to_polars_schema` handles Optionals and lists of nested models.

In [1]:
from typing import Optional, List, Union
from pydantic import BaseModel
import polars as pl
from poldantic import to_polars_schema

In [2]:
class Location(BaseModel):
    lat: float
    lon: float

In [3]:
class Event(BaseModel):
    id: int | str
    name: Optional[str]
    location: Optional[Location]
    tags: Optional[List[str]]
    data: Union[str, None]

In [4]:
schema = to_polars_schema(Event)
print(schema)

{'id': Object, 'name': String, 'location': Struct({'lat': Float64, 'lon': Float64}), 'tags': List(String), 'data': String}


In [6]:
# Construct a Polars DataFrame using this schema
data = [
    {"id": 1, "name": "Launch", "location": {"lat": 40.0, "lon": -70.0}, "tags": ["rocket"], "data": "payload"},
    {"id": '2', "name": None, "location": None, "tags": None, "data": None},
]
df = pl.DataFrame(data, schema=schema)
df

id,name,location,tags,data
object,str,struct[2],list[str],str
1,"""Launch""","{40.0,-70.0}","[""rocket""]","""payload"""
2,null,null,null,null
